In [1]:
!pip install pyxlsb
!pip install pandas_market_calendars
!pip install stable-baselines3
!pip install shimmy>=0.2.1

In [2]:
import pandas as pd

# Define the S3 path to your .xlsx file
file_path = '../data'

# Read the Excel file directly into a pandas DataFrame
df_purchase = pd.read_excel(file_path + "base2023_compra.xlsb", engine="pyxlsb")
df_purchase_2 = pd.read_excel(file_path + "base2023_compra_2.xlsb", engine="pyxlsb")

# Now you can work with the DataFrame 'data'
print(df_purchase.head())

/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/fsspec/registry.py:273: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


  Cód. Cliente  Dt. Operação Tipo Operação Cód. Título Cód. Corretora  \
0    Cliente 1         45057             C       GGBR4           FLOW   
1    Cliente 2         45057             C      SANB11           GARA   
2    Cliente 3         45057             C       PRIO3           CITI   
3    Cliente 4         45225             C       WEGE3           XPIN   
4    Cliente 5         44964             C       VALE3           INDU   

   Quantidade      Preço  Valor Líquido  Dt. Liquidação Tipo  
0         112  24.120000       -2704.11           45061   AV  
1           6  30.060000        -180.58           45061   AV  
2         900  34.540000      -31115.09           45061   AV  
3         100  31.800600       -3180.06           45229   AV  
4         900  87.354911      -78619.42           44966   AV  


In [23]:
# change the date columns to pandas datetime
df_purchase_datetime = df_purchase.copy(deep=True)
df_purchase_2_datetime = df_purchase_2.copy(deep=True)

df_purchase_datetime['Dt. Operação'] = pd.to_datetime(df_purchase_datetime['Dt. Operação'], unit='D', origin='1899-12-30')
df_purchase_2_datetime['Dt. Operação'] = pd.to_datetime(df_purchase_2_datetime['Dt. Operação'], format="%d/%m/%Y")

df_purchase_2_datetime.rename(columns={"Vencimento": "Dt. Liquidação"}, inplace=True)

df_purchase_datetime['Dt. Liquidação'] = pd.to_datetime(df_purchase_datetime['Dt. Liquidação'], unit='D', origin='1899-12-30')
df_purchase_2_datetime['Dt. Liquidação'] = pd.to_datetime(df_purchase_2_datetime['Dt. Liquidação'], errors='coerce', format="%d/%m/%Y")

df_purchases = pd.concat([df_purchase_datetime, df_purchase_2_datetime], ignore_index=True)
df_purchases = df_purchases.sort_values(by=["Cód. Cliente", "Dt. Operação", "Cód. Título", "Cód. Corretora"]).reset_index()
pd.set_option('display.max_columns', None)

In [24]:
df_sales = pd.read_excel(file_path + "base2023_venda.xlsb", engine="pyxlsb")

df_sales_datetime = df_sales.copy(deep=True)
df_sales_datetime['Dt. Operação'] = pd.to_datetime(df_sales_datetime['Dt. Operação'], unit='D', origin='1899-12-30')
df_sales_datetime['Vencimento'] = pd.to_datetime(df_sales_datetime['Vencimento'], unit='D', origin='1899-12-30')
df_sales_datetime = df_sales_datetime.sort_values(by=["Cód. Cliente", "Dt. Operação", "Cód. Título", "Cód. Corretora"]).reset_index()

df_filtered_purchases = df_purchases.copy(deep=True)
df_filtered_sales = df_sales_datetime.copy(deep=True)

df_filtered_purchases = df_filtered_purchases.drop(["index", "Tipo Operação", "Valor Líquido", "Dt. Liquidação", "Tipo"], axis=1)
df_filtered_sales = df_filtered_sales.drop(["index", "Tipo Operação", "Valor Líquido", "Tipo"], axis=1)

In [25]:
import pandas_market_calendars as mcal
brazil_calendar = mcal.get_calendar('BMF')

def calculate_du(x):
  business_days = brazil_calendar.valid_days(x["Dt. Operação"], x["Vencimento"])
  du = len(business_days) - 1
  return du

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
df_filtered_sales["du"] = df_filtered_sales.apply(calculate_du, axis=1)

df_filtered_sales_du = df_filtered_sales.copy(deep=True)
df_filtered_sales_du = df_filtered_sales[(df_filtered_sales["du"] > 0)].sort_values(by=['Cód. Cliente', 'Dt. Operação', 'Cód. Título', 'Cód. Corretora'])
display(df_filtered_sales_du.head())

,Chave,Cód. Cliente,Dt. Operação,Cód. Título,Cód. Corretora,Quantidade,Preço,Vencimento,DI,du
0,4721A,Cliente 1,2023-01-05,XPBR31,XPIN,4470,76.450544,2023-03-21,0.136781,51
1,19007A,Cliente 1,2023-01-05,XPBR31,XPIN,35530,76.440559,2023-03-21,0.136781,51
2,20955A,Cliente 1,2023-01-05,XPBR31,XPIN,5440,76.069360,2023-03-06,0.136704,40
3,30031A,Cliente 1,2023-01-05,XPBR31,XPIN,14560,76.079336,2023-03-06,0.136704,40
4,18899A,Cliente 1,2023-01-19,XPBR31,XPIN,41230,86.807599,2023-02-22,0.136585,22


In [26]:
from tqdm import tqdm
filtered_dfs = []

for index, row in tqdm(df_filtered_sales_du.sample(100).iterrows()):
  available_purchases = df_filtered_purchases[
    (df_filtered_purchases['Cód. Cliente'] == row['Cód. Cliente']) &
    (df_filtered_purchases['Dt. Operação'] == row['Dt. Operação']) &
    (df_filtered_purchases['Cód. Corretora'] == row['Cód. Corretora']) &
    (df_filtered_purchases['Cód. Título'] == row['Cód. Título'])
  ]

  if not available_purchases.empty:
    filtered_dfs.append({
        'sale': row.copy(deep=True),
        'purchase': available_purchases.copy(deep=True)
    })

100it [00:32,  3.08it/s]


# Ambiente

In [27]:
from gym import Env
from gym.spaces import Discrete, Box, Dict
import numpy as np
import random
import os

In [28]:
def calculate_current_price(purchases_quantities, purchases): # ((preco, quantidade), (preco, quantidade), ...)
    total_price = 0
    total_quantity = 0

    for purchase in range(len(purchases_quantities)):
        total_price += purchases.iloc[purchase]["Preço"] * purchases_quantities[purchase]
        total_quantity += purchases_quantities[purchase]

    if total_quantity == 0:
      return 0

    return total_price / total_quantity

In [29]:
def calculate_ideal_price(sale_price, du, di):
  #pi=pv/(di+1)^du/252
  ideal_price = sale_price / ((di + 1) ** (du / 252))
  return ideal_price

In [30]:
def calculate_cdi(du, purchase_price, sale_price, di):
    if purchase_price == 0:
      return 0
    rent = ((sale_price / purchase_price) - 1)
    annual_rent = ((1 + rent) ** (252 / du)) - 1

    cdi = (annual_rent / di) * 100

    return cdi

In [31]:
import math
def step_reward(sale_price, ideal_price, current_price, old_price, is_valid):

    if not is_valid:
        return -1e3

    reward = 0

    if (ideal_price - current_price) > (ideal_price - old_price):
      reward = -100
    elif (ideal_price - current_price) < (ideal_price - old_price):
      reward = 100

    return reward


def episode_reward(di, du, current_price, sale_price):
    cdi = calculate_cdi(du, current_price, sale_price, di)

    if (99.0 <= cdi <= 101.0):
        return 1e4
    elif (101.0 < cdi <= 110.0):
        return 1e2
    elif (80.0 <= cdi <= 98.9):
        return 1e-1
    else:
        return 1e-6

In [32]:
def update_dataframes(current_sale_index, dataframes, purchases_quantities):
  # Critérios do 'current_episode'
  current_sale = dataframes[current_sale_index]['sale']

  # Loop através dos 'episodes' começando do próximo após 'current_episode'
  for sale_index in range(current_sale_index + 1, len(dataframes)):
      sale = filtered_dfs[sale_index]['sale']

      # Verifica se os critérios são atendidos
      criterios = (sale['Cód. Cliente'] == current_sale['Cód. Cliente'] and
                  sale['Dt. Operação'] == current_sale['Dt. Operação'] and
                  sale['Cód. Título'] == current_sale['Cód. Título'] and
                  sale['Cód. Corretora'] == current_sale['Cód. Corretora'])

      if not criterios:
          break

      purchases = dataframes[sale_index]['purchase']
      # Atualize as quantidades baseado no dicionário index_quantidade
      for i in range(len(purchases_quantities)):
        purchases.iloc[i, purchases.columns.get_loc("Quantidade")] -= purchases_quantities[i]

  return dataframes
  # Note que o loop 'for' verifica todos os episódios, mesmo que não cumpram os critérios,
  # mas apenas executa a atualização de quantidade se os critérios forem atendidos.

In [33]:
observation_data = [
  "sale_price",
  "sale_quantity",
  "combination_current_price",
  "purchase_price",
  "purchase_quantity",
  "ideal_price",
  "purchase_combined_quantity"
]

In [34]:
from copy import deepcopy
class Environment(Env):
    def __init__(self):
        self.filtered_dfs = deepcopy(filtered_dfs)

        #self.actions = [1, 0.75, 0.50, 0.25, 0, -0.25, -0.50, -0.75, -1]
        self.actions = [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0, -0.1, -0.2, -0.3, -0.4, -0.5, -0.6, -0.7, -0.8, -0.9, -1]
        self.observation_space = Box(low=-np.inf, high=np.inf, shape=(len(observation_data),), dtype=np.float64)

        self.quantities_price = [(0,0)]
        #self.quantities_index = np.zeros(1)

        self.all_quantities = [np.zeros(x['purchase'].shape[0]) for x in filtered_dfs]

        self.cdis = np.zeros(len(self.filtered_dfs))
        self.best_cdis = np.zeros(len(self.filtered_dfs))

        # Quantas ações podem ser tomadas
        self.action_space = Discrete(len(self.actions))

        self.current_episode = 0
        self.current_step = 0

        self.max_steps = 0

        self.du = 0
        self.di = 0

        self.state = None

    # atualizar as quantidades de compra e venda de acordo com a ação tomada no estado S
    def update_data(self, action):
      is_valid = True
      current_purchase_index = self.current_step % self.purchases.shape[0]


      if action < 0:
        if self.all_quantities[self.current_episode][current_purchase_index] == 0:
            is_valid = False

        quantity_of_purchase = int(action * self.all_quantities[self.current_episode][current_purchase_index])
      else:
        quantity_of_purchase = int(action * self.purchases.iloc[current_purchase_index]['Quantidade'])

      if quantity_of_purchase < 0:
        if abs(quantity_of_purchase) > self.all_quantities[self.current_episode][current_purchase_index]:
          quantity_of_purchase = -1 * self.all_quantities[self.current_episode][current_purchase_index]

      # se ele estiver tentando pegar mais do que tem na quantidade de venda
      # ele pega o que tem na quantidade de venda
      if quantity_of_purchase - self.sale["Quantidade"] > 0:
        quantity_of_purchase = self.sale["Quantidade"]

      self.all_quantities[self.current_episode][current_purchase_index] += quantity_of_purchase

      self.sale["Quantidade"] = self.sale["Quantidade"] - quantity_of_purchase
      # print("action ", action)
      #print("compras", self.purchases.iloc[current_purchase_index]["Quantidade"])
      self.purchases.iloc[current_purchase_index, self.purchases.columns.get_loc("Quantidade")] -= quantity_of_purchase
      # print("acao: ", action)
      # print("quantity_of_purchase: ", quantity_of_purchase)
      # print("current purchase ", current_purchase_index)
      # print("Sales: ", self.sale["Quantidade"])

      return quantity_of_purchase, is_valid



    def step(self, action):
        done = False
        quantidade_compra = self.actions[action]

        quantidade_ajustada, is_valid = self.update_data(quantidade_compra)

        preco_medio_antigo = self.state[2]
        preco_venda = self.state[0]

        self.quantities_price.append((self.state[3], quantidade_ajustada))

        preco_medio_at = calculate_current_price(self.all_quantities[self.current_episode], self.purchases)

        # calcular recompensa
        # reward = calculate_reward(self.du, self.di, preco_medio_at, preco_venda, is_valid)
        reward = step_reward(self.state[0], self.state[5], preco_medio_at, preco_medio_antigo, is_valid)

        # atualizar variaveis
        self.current_step += 1

        # atualizar estado
        self.state = self.__get_observation__()

        # condicao de parada
        cdi = calculate_cdi(self.du, preco_medio_at, self.state[0], self.di)
        if (self.state[1] == 0
             and 100 <= cdi <= 104):
          reward += 1000
          done = True
          self.cdis[self.current_episode] = cdi
          self.best_cdis[self.current_episode] = cdi

        elif self.current_step >= self.max_steps:
          original_sale_quantity = filtered_dfs[self.current_episode]['sale']['Quantidade']
          sale_completion = sum(self.all_quantities[self.current_episode]) / original_sale_quantity

          reward += -100 + 10**sale_completion
          if sum(self.all_quantities[self.current_episode]) == 0:
            reward = -1e3
          done = True
          self.cdis[self.current_episode] = cdi

        #print("reward: ", reward)

        if done:
          #print("quantities_index: ", self.all_quantities[self.current_episode])
          #print(self.state)
          #print("episodio: ", self.current_episode)
          #print("final reward: ", reward)
          self.filtered_dfs = update_dataframes(self.current_episode, self.filtered_dfs, self.all_quantities[self.current_episode])
          self.current_episode += 1
          reward += episode_reward(self.du, self.di, preco_medio_at, preco_venda)

        info = {}
        return self.state.reshape(len(observation_data),), reward, done, info

    def render(self, mode):
        # Pode ser implementado para "Exibir" o cenário
        # tipo o gráfico do grid do gridworld
        # opcional
        pass

    def reset(self):
        self.current_episode = self.current_episode % len(self.filtered_dfs)
        self.purchases = self.filtered_dfs[self.current_episode]['purchase']
        self.sale = self.filtered_dfs[self.current_episode]['sale']

        self.current_step = 0
        self.quantities_price = [(0, 0)]
        self.state = self.__get_observation__()
        #self.quantities_index = np.zeros(self.purchases.shape[0])
        return self.state.reshape(len(observation_data), )

    def __get_observation__(self):
        quantidade_linhas_compra = self.purchases.shape[0]
        index_compra_atual = self.current_step % quantidade_linhas_compra

        self.di = self.sale['DI']
        preco_venda = self.sale['Preço']
        self.du = self.sale['du']

        preco_ideal = calculate_ideal_price(preco_venda, self.du, self.di) #implentar

        quantidade_venda = self.sale['Quantidade']

        if quantidade_venda > 100:
          quantidade_venda /= 100
          if quantidade_venda > 100:
            quantidade_venda /= 10

        preco_compra = self.purchases.iloc[index_compra_atual]['Preço']
        quantidade_compra = self.purchases.iloc[index_compra_atual]['Quantidade']

        if quantidade_compra > 100:
          quantidade_compra /= 100
          if quantidade_compra > 100:
            quantidade_compra /= 10

        if self.current_step == 0:
          preco_medio_at = preco_compra
        else:
            preco_medio_at = calculate_current_price(self.all_quantities[self.current_episode], self.purchases)

        purchase_combined_quantity = self.all_quantities[self.current_episode][index_compra_atual]
        if purchase_combined_quantity > 100:
          purchase_combined_quantity /= 100
          if purchase_combined_quantity > 100:
            purchase_combined_quantity /= 10

        self.max_steps = quantidade_linhas_compra * max_steps

        return np.array([preco_venda, quantidade_venda, preco_medio_at, preco_compra, quantidade_compra, preco_ideal, purchase_combined_quantity]).reshape(len(observation_data),)


# Agente

In [35]:
# Hiperparâmetros
max_steps = 5
learning_rate = 0.01
nb_steps = 1000000
neurons_hidden_layer = 24
hidden_layer_activation = 'relu'

In [36]:
!pip install keras-rl2

In [37]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from keras import __version__
tf.keras.__version__ = __version__

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
from rl.callbacks import Callback

env = Environment()
states = (len(observation_data), )
actions = env.action_space.n

def build_model():
    model = Sequential()
    model.add(Flatten(input_shape=(1, len(observation_data))))
    model.add(Dense(neurons_hidden_layer, activation=hidden_layer_activation, input_shape=states))
    model.add(Dense(neurons_hidden_layer, activation=hidden_layer_activation))
    model.add(Dense(neurons_hidden_layer, activation=hidden_layer_activation))
    model.add(Dense(actions, activation='softmax'))
    return model

def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=100, target_model_update=1e-2)
    return dqn

In [38]:
model = build_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 7)                 0         
                                                                 
 dense_4 (Dense)             (None, 24)                192       
                                                                 
 dense_5 (Dense)             (None, 24)                600       
                                                                 
 dense_6 (Dense)             (None, 24)                600       
                                                                 
 dense_7 (Dense)             (None, 21)                525       
                                                                 
Total params: 1917 (7.49 KB)
Trainable params: 1917 (7.49 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [39]:
import matplotlib.pyplot as plt

class PlotRewardsCallback(Callback):
    def __init__(self):
        super(PlotRewardsCallback, self).__init__()
        # Initialize lists to store episode numbers and rewards
        self.episodes = []
        self.rewards = []

    def on_episode_end(self, episode, logs={}):
        # Append episode number and reward
        self.episodes.append(episode)
        self.rewards.append(logs['episode_reward'])

        # Plot or update the plot
        plt.figure(figsize=(10, 5))
        plt.plot(self.episodes, self.rewards, marker='o', linestyle='-')
        plt.title('Episode Rewards')
        plt.xlabel('Episode')
        plt.ylabel('Reward')
        plt.show()

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

dqn = build_agent(model, actions)
dqn.compile(tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate), metrics=['mse'])
plot_rewards_callback = PlotRewardsCallback()
dqn.fit(env, nb_steps=nb_steps, visualize=True, verbose=1, nb_max_episode_steps=None)
env.cdis

2024-03-29 16:04:03.932951: W tensorflow/c/c_api.cc:305] Operation '{name:'dense_6_1/kernel/Assign' id:983 op device:{requested: '', assigned: ''} def:{{{node dense_6_1/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_6_1/kernel, dense_6_1/kernel/Initializer/stateless_random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Training for 1000000 steps ...
Interval 1 (0 steps performed)


/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2024-03-29 16:04:04.309907: W tensorflow/c/c_api.cc:305] Operation '{name:'dense_7/Softmax' id:919 op device:{requested: '', assigned: ''} def:{{{node dense_7/Softmax}} = Softmax[T=DT_FLOAT, _has_manual_control_dependencies=true](dense_7/BiasAdd)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2024-03-29 16:04:04.374550: W tensorflow/c/c_api.cc:305] Operation '{name:'count_5/Assign' id:1186 op device:{requested: '', assigned: ''} def:{{{node count_5/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=

  100/10000 [..............................] - ETA: 2:54 - reward: -382.0000

2024-03-29 16:04:06.317046: W tensorflow/c/c_api.cc:305] Operation '{name:'dense_7_1/Softmax' id:1018 op device:{requested: '', assigned: ''} def:{{{node dense_7_1/Softmax}} = Softmax[T=DT_FLOAT, _has_manual_control_dependencies=true](dense_7_1/BiasAdd)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2024-03-29 16:04:06.530568: W tensorflow/c/c_api.cc:305] Operation '{name:'loss_7/AddN' id:1301 op device:{requested: '', assigned: ''} def:{{{node loss_7/AddN}} = AddN[N=2, T=DT_FLOAT, _has_manual_control_dependencies=true](loss_7/mul, loss_7/mul_1)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2024-03-29 16:04:06.595626: W tensorflow/c/c_api.cc:305] Operation '{

10000/10000 [==============================] - 260s 26ms/step - reward: -380.4834
48 episodes - episode_reward: -78240.283 [-133990.000, 1000.000] - loss: 201450.492 - mse: 19185.802 - mean_q: 1.000

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 290s 29ms/step - reward: -377.6576
45 episodes - episode_reward: -84999.467 [-288790.000, -29800.000] - loss: 190502.875 - mse: 18143.162 - mean_q: 1.000

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 280s 28ms/step - reward: -330.6263
51 episodes - episode_reward: -64297.310 [-113290.000, 1200.000] - loss: 187995.938 - mse: 17904.398 - mean_q: 1.000

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 272s 27ms/step - reward: -293.0785
45 episodes - episode_reward: -64955.227 [-269290.000, -3090.327] - loss: 177327.656 - mse: 16888.412 - mean_q: 1.000

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 278s 28ms/

array([ 9.94169376e+01, -4.66013111e+02, -3.50782554e+01, -2.32025458e+02,
        1.98025789e+02, -1.73241714e+02,  1.02986275e+02,  1.03041608e+02,
       -2.25301184e+02,  1.03016322e+02,  1.00582081e+02, -6.28437787e+01,
       -1.46374917e+02,  3.21765136e-01, -4.91408008e+02, -6.24507622e+01,
       -5.38455564e+02,  1.03239368e+02, -6.02126519e+02, -5.57578031e+01,
       -5.51722484e+02, -5.70175383e+01,  1.03830771e+02, -1.48255127e+02,
       -3.87368709e+02, -5.60685046e+02, -7.81467485e+01, -8.83846718e+01,
       -1.20564335e+02, -3.38000497e+01, -3.77261798e+02, -6.86477253e+02,
        1.00945232e+02,  1.00514897e+02, -5.74298601e+01, -3.46897826e+02,
       -5.13379762e+02, -1.89762059e+02, -3.09660803e+01, -1.96616974e+02,
       -1.55514945e+02, -1.54797507e+01, -1.15894213e+02,  1.00253014e+02,
       -1.56511587e+02, -5.04638040e+02,  2.10772560e+01,  1.02265589e+02,
       -4.63024202e+02, -9.26787118e+01, -2.07881050e+02, -2.88718802e+02,
        1.03056819e+02, -

In [ ]:
# [preco_venda, quantidade_venda, preco_medio_at, preco_compra, quantidade_compra, preco_ideal, purchase_combined_quantity]

In [47]:

count = 0
for i in range(len(env.best_cdis)):
  if env.best_cdis[i]:
    count += 1

print(f"Acertos: {(count / len(env.cdis)) * 100}")

Acertos: 24.0
